In [1]:
import os
import sys
import json

curdir = os.getcwd()
while 'config.json' not in os.listdir(curdir):
    curdir = os.path.dirname(curdir)
with open(os.path.join(curdir,'config.json'), 'r') as f:
    dataset = json.load(f)['dataset']

curdir = os.getcwd()
while 'filepathhelper.py' not in os.listdir(curdir):
       curdir = os.path.dirname(curdir)
sys.path.append(curdir)
import filepathhelper

In [2]:
from neo4j import GraphDatabase
import pandas as pd
import numpy as np
# from tqdm import tqdm
from tqdm import tqdm_notebook as tqdm

# Initialize DB and read csv

In [3]:
uri = "bolt://localhost:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "jrdb"))

In [4]:
def create_issue(tx,issuekey):
    tx.run("CREATE (:Issue {name: $issuekey})",issuekey=issuekey)
def create_issue_relation(tx,issueA,issueB,relation):
    relation = relation.replace(" ","_")
    relation = relation.replace("-","_")
    tx.run("MATCH (a:Issue),(b:Issue)"
           "WHERE a.name = $issueA AND b.name = $issueB AND NOT EXISTS((a)-[:"+relation+"]->(b))"
           "CREATE (a)-[:"+relation+" {weight: 1}]->(b)",issueA=issueA,issueB=issueB)
def create_and_return_greeting(tx, message):
    result = tx.run("CREATE (a:Greeting) "
                        "SET a.message = $message "
                        "RETURN a.message + ', from node ' + id(a)", message=message)
    return result.single()[0]

In [5]:
issuelink = pd.read_csv(filepathhelper.path(dataset,'issuelink.csv'),sep=';')
trainissue = pd.read_csv(filepathhelper.path(dataset,'trainissuekey.csv'))

In [6]:
issuelink = issuelink[(issuelink['u'].isin(trainissue['issuekey']))]
issuelink = issuelink[(issuelink['v'].isin(trainissue['issuekey']))]

# Create issuekey

In [7]:
allissue = set(issuelink['u'].unique())
allissue.update(issuelink['v'].unique())
len(allissue)

8131

In [8]:
with driver.session() as session:
#     session.read_transaction(print_friends_of, "Alice")
    for i in tqdm(allissue):
        session.write_transaction(create_issue, i)


/home/noppadolass/.local/lib/python3.7/site-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


# Create Relationship

In [9]:
with driver.session() as session:
    for key,issue in tqdm(issuelink.iterrows(),total=issuelink.shape[0]):
        u = issue['u']
        v = issue['v']
        session.write_transaction(create_issue_relation, u,v,issue['relation'])


/home/noppadolass/.local/lib/python3.7/site-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  
